## Let's first import the necessary libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [3]:
df = pd.read_csv('./dataset/Parco_Veicoli.csv')
df

/var/folders/r2/1zz301w12bv657wd0zmshw_00000gn/T/ipykernel_23741/1160917084.py:1: DtypeWarning: Columns (7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/Parco_Veicoli.csv')


,PROGRESSIVO,TIPO_TARGA,UTILIZZO,SPECIALITA,RIMORCHIABILITA,CILIDRATA,ALIMENTAZIONE,PORTATA,NUM_POSTI,TIPO_IMPIANTO,DISISTALLAZIONE_IMPIANTO,KW,ANNO_IMMATRICOLAZIONE,NUM_ASSI,EURO,CO2,PESO_COMPLESSIVO,CAP_RESIDENZA_PROPR,PROV_RESIDENZA
0,1,3,4,NaN,N,NaN,NaN,800.0,000,NaN,NaN,NaN,1998,2.0,0,NaN,1200.0,89046.0,RC
1,2,1,PV,NaN,NaN,1698.0,GA,385.0,005,NaN,NaN,51.0,1998,2.0,2,0.0,1510.0,88046.0,CZ
2,3,1,5,NaN,NaN,14618.0,GA,16730.0,003,NaN,NaN,320.0,1998,4.0,2,0.0,32000.0,87036.0,CS
3,4,1,5,NaN,NaN,13798.0,GA,11450.0,002,NaN,NaN,309.0,1998,3.0,2,0.0,26000.0,88060.0,CZ
4,5,1,0,NaN,NaN,1998.0,BE,500.0,005,NaN,NaN,137.0,1998,2.0,2,0.0,2010.0,87036.0,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433489,1433490,1,PV,NaN,S,NaN,ELE,00515,5.0,NaN,NaN,127.0,2023,2.0,NE,NaN,2895.0,88040.0,CZ
1433490,1433491,1,6,NaN,N,12902.0,G,13100,3.0,NaN,NaN,300.0,2008,3.0,4,NaN,26000.0,88100.0,CZ
1433491,1433492,1,NR,PIA,S,2488.0,G,NaN,3.0,NaN,NaN,81.0,2011,2.0,4,NaN,3500.0,87050.0,CS
1433492,1433493,1,NC,NaN,S,12740.0,G,04200,55.0,NaN,NaN,324.0,2013,2.0,5,NaN,18000.0,89824.0,VV


## Now it's time to prepare the data for the model

In [4]:
# change the column names into lowercase
df.columns = df.columns.str.lower()
df

,progressivo,tipo_targa,utilizzo,specialita,rimorchiabilita,cilidrata,alimentazione,portata,num_posti,tipo_impianto,disistallazione_impianto,kw,anno_immatricolazione,num_assi,euro,co2,peso_complessivo,cap_residenza_propr,prov_residenza
0,1,3,4,NaN,N,NaN,NaN,800.0,000,NaN,NaN,NaN,1998,2.0,0,NaN,1200.0,89046.0,RC
1,2,1,PV,NaN,NaN,1698.0,GA,385.0,005,NaN,NaN,51.0,1998,2.0,2,0.0,1510.0,88046.0,CZ
2,3,1,5,NaN,NaN,14618.0,GA,16730.0,003,NaN,NaN,320.0,1998,4.0,2,0.0,32000.0,87036.0,CS
3,4,1,5,NaN,NaN,13798.0,GA,11450.0,002,NaN,NaN,309.0,1998,3.0,2,0.0,26000.0,88060.0,CZ
4,5,1,0,NaN,NaN,1998.0,BE,500.0,005,NaN,NaN,137.0,1998,2.0,2,0.0,2010.0,87036.0,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433489,1433490,1,PV,NaN,S,NaN,ELE,00515,5.0,NaN,NaN,127.0,2023,2.0,NE,NaN,2895.0,88040.0,CZ
1433490,1433491,1,6,NaN,N,12902.0,G,13100,3.0,NaN,NaN,300.0,2008,3.0,4,NaN,26000.0,88100.0,CZ
1433491,1433492,1,NR,PIA,S,2488.0,G,NaN,3.0,NaN,NaN,81.0,2011,2.0,4,NaN,3500.0,87050.0,CS
1433492,1433493,1,NC,NaN,S,12740.0,G,04200,55.0,NaN,NaN,324.0,2013,2.0,5,NaN,18000.0,89824.0,VV


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433494 entries, 0 to 1433493
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   progressivo               1433494 non-null  int64  
 1   tipo_targa                1433494 non-null  int64  
 2   utilizzo                  1433475 non-null  object 
 3   specialita                19609 non-null    object 
 4   rimorchiabilita           921586 non-null   object 
 5   cilidrata                 1408879 non-null  float64
 6   alimentazione             1411074 non-null  object 
 7   portata                   1431120 non-null  object 
 8   num_posti                 1431699 non-null  object 
 9   tipo_impianto             60800 non-null    object 
 10  disistallazione_impianto  53 non-null       object 
 11  kw                        1411419 non-null  float64
 12  anno_immatricolazione     1433494 non-null  int64  
 13  num_assi                  1

In [6]:
df.shape

(1433494, 19)

In [7]:
df = df[~((df["co2"] == 0) & (df["alimentazione"] != "ELE"))].copy()

df.loc[df["alimentazione"] == "ELE", "co2"] = df.loc[df["alimentazione"] == "ELE", "co2"].fillna(0)

df.dropna(subset=["prov_residenza", "alimentazione", "anno_immatricolazione", "co2"], inplace=True)

df.loc[df["alimentazione"] == "ELE", "co2"]

866548     0.0
916685     0.0
1016953    0.0
1021838    0.0
1028143    0.0
          ... 
1433342    0.0
1433343    0.0
1433398    0.0
1433409    0.0
1433489    0.0
Name: co2, Length: 1920, dtype: float64

In [8]:
df = df[["prov_residenza", "anno_immatricolazione", "co2"]]

df

,prov_residenza,anno_immatricolazione,co2
16,CZ,1998,1960.0
1102,CS,1998,1640.0
1105,CS,1998,1640.0
1117,KR,1998,1600.0
1120,CS,1998,1640.0
...,...,...,...
1433483,RC,2011,1100.0
1433486,KR,2024,1050.0
1433487,RC,2019,1110.0
1433488,CS,2020,520.0


In [9]:
df = df.groupby(["prov_residenza", "anno_immatricolazione"])["co2"].mean().round(0).astype(int).reset_index()

df

,prov_residenza,anno_immatricolazione,co2
0,CS,1998,1604
1,CS,1999,1573
2,CS,2000,1544
3,CS,2001,1532
4,CS,2002,1521
...,...,...,...
130,VV,2020,1132
131,VV,2021,1243
132,VV,2022,1188
133,VV,2023,1169


## Define the model